In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from random import randint
import numpy as np
import utils
import readin_v5
import os
import time

In [2]:
class yololike(nn.Module):
    
    def __init__(self):
        super(yololike,self).__init__()
    
        # layer1, 3*360*640  --> 16*360*640 
        self.conv1=nn.Conv2d(3,16,kernel_size=3,padding=1)
        # maxpool,16*360*640 --> 16*180*320
        self.pool1=nn.MaxPool2d(2,2)
        self.norm1=nn.BatchNorm2d(16)
    
        # layer2, 16*180*320 --> 32*180*320 
        self.conv2=nn.Conv2d(16,32,kernel_size=3,padding=1)
        # maxpool,32*180*320 --> 32*90*160
        self.pool2=nn.MaxPool2d(2,2)
        self.norm2=nn.BatchNorm2d(32)
        
        # layer3, 32*90*160 --> 64*90*160 
        self.conv3=nn.Conv2d(32,64,kernel_size=3,padding=1)
        # maxpool,64*90*160 --> 64*45*80
        self.pool3=nn.MaxPool2d(2,2)
        self.norm3=nn.BatchNorm2d(64)
        
        # layer4, 64*45*80 --> 128*45*80
        self.conv4=nn.Conv2d(64,128,kernel_size=3,padding=1)
        # maxpool,128*45*80 --> 128*23*40
        self.pool4=nn.MaxPool2d(2,2,padding=[1,0])
        self.norm4=nn.BatchNorm2d(128)
        
        # layer5, 128*23*40 --> 256*23*40
        self.conv5=nn.Conv2d(128,256,kernel_size=3,padding=1)
        # maxpool,256*23*40 --> 256*12*20
        self.pool5=nn.MaxPool2d(2,2,padding=[1,0])
        self.norm5=nn.BatchNorm2d(256)
        
        # layer6, 256*12*20 --> 512*12*20 
        self.conv6=nn.Conv2d(256,512,kernel_size=3,padding=1)
        # maxpool,512*12*20 --> 512*6*10?
        self.pool6=nn.MaxPool2d(2,2)
        self.norm6=nn.BatchNorm2d(512)
        
        # layer7, 512* --> 1024*
        self.conv7=nn.Conv2d(512,1024,kernel_size=3,padding=1)
        self.norm7=nn.BatchNorm2d(1024)
        
        ##################################
        # Location Network, the same as DeepZ
        ##################################
        
        # layer8, 1024* --> 512* 
        self.conv8=nn.Conv2d(1024,512,kernel_size=3,padding=1)
        self.norm8=nn.BatchNorm2d(512)
        
        # layer9, 512* --> 125*6*10=7500
        self.conv9=nn.Conv2d(512,125,kernel_size=1,padding=0)
        
        # layer10-a, region
        self.line10=nn.Linear(7500,84)
        
    def forward(self,x):
        
        # Layer1,activation=leaky
        x=self.conv1(x)
        x=F.leaky_relu(x)
        x=self.pool1(x)
        x=self.norm1(x)
        
        # layer2,activation=leaky
        x=self.conv2(x)
        x=F.leaky_relu(x)
        x=self.pool2(x)
        x=self.norm2(x)
        
        # layer3,activation=leaky
        x=self.conv3(x)
        x=F.leaky_relu(x)
        x=self.pool3(x)
        x=self.norm3(x)
        
        # layer4,activation=leaky
        x=self.conv4(x)
        x=F.leaky_relu(x)
        x=self.pool4(x)
        x=self.norm4(x)
        
        # layer5,activation=leaky
        x=self.conv5(x)
        x=F.leaky_relu(x)
        x=self.pool5(x)
        x=self.norm5(x)
        #print(x.size())
        
        # layer6,activation=leaky
        x=self.conv6(x)
        x=F.leaky_relu(x)
        x=self.pool6(x)
        x=self.norm6(x)
        #print(x.size())
        # layer7,activation=leaky
        x=self.conv7(x)
        x=F.leaky_relu(x)
        x=self.norm7(x)
        #print(x.size())
        # layer8,activation=leaky
        x=self.conv8(x)
        x=F.leaky_relu(x)
        x=self.norm8(x)
        #print(x.size())
        # layer9,activation=leaky
        x=self.conv9(x)
        x=F.leaky_relu(x)
        #print(x.size())
        
        # layer10,activation=leaky
        x=x.view(-1,7500)
        x=self.line10(x)
        
        return(x)

In [3]:
def cal_iou(infer,target):
    x=infer[0]
    y=infer[1]
    w=infer[2]
    h=infer[3]
    xt=target[0]
    yt=target[1]
    wt=target[2]
    ht=target[3]
    # case 0: when contained by the target
    if ((x+w)>=xt) and ((y+h)>=yt) and ((x+w)<=xt+wt) and ((y+h)<=yt+ht) and (x>=xt) and (y>=yt) and (x<=xt+wt) and (y<=yt+ht):
        same=w*h
        iou=same/(wt*ht)
    # case 1: when right upper part is the same
    elif ((x+w)>=xt) and ((y+h)>=yt) and ((x+w)<=xt+wt) and ((y+h)<=yt+ht):
        same=(x+w-xt)*(y+h-yt)
        iou=same/(w*h+wt*ht-same)
    # case 2: right down part
    elif ((x+w)>=xt) and (y>=yt) and ((x+w)<=xt+wt) and (y<=yt+ht):
        same=(x+w-xt)*(yt+ht-y)
        iou=same/(w*h+wt*ht-same)
    # case 3: left upper part
    elif (x>=xt) and ((y+h)>=yt) and (x<=xt+wt) and ((y+h)<=yt+ht):
        same=(xt+wt-x)*(y+h-yt)
        iou=same/(w*h+wt*ht-same)
    # case 4: left down part
    elif (x>=xt) and (y>=yt) and (x<=xt+wt) and (y<=yt+ht):
        same=(xt+wt-x)*(yt+ht-y)
        iou=same/(w*h+wt*ht-same)
    # case 5: contain the target
    elif ((xt+wt)>=x) and ((yt+ht)>=y) and ((xt+wt)<=x+w) and ((yt+ht)<=y+h) and (xt>=x) and (yt>=y) and (xt<=x+w) and (yt<=y+h):
        same=wt*ht
        iou=same/(w*h)
    else :
        same=0.0
        iou=0.0
    del x,y,w,h,xt,yt,wt,ht,same
    return iou

In [4]:
def get_iou(out_prob,real_label,bs):
    iou_count=0.0
    for i in range(bs):
        iou_count=iou_count+cal_iou(out_prob[i],real_label[i])
    return(iou_count/bs)

In [5]:
def get_accu(out_prob,real_label,bs):
    count=0
    for i in range(bs):
        temp=0.0
        maxp=0
        for j in range(80):
            if temp<out_prob[i,j]:
                temp=out_prob[i,j]
                maxp=j
        if maxp==real_label[i,0]:
            count=count+1
    return(count)

In [9]:
mydir=os.getcwd()
train_data_dir=mydir+"\\train_data"
train_label_dir=mydir+"\\train_label"
small_data_dir=mydir+"\\small_data"
small_label_dir=mydir+"\\small_label"

net = yololike()
origname="yolo2tiny_v2_6000"
record=open(origname+"_record.txt",mode='a')

bs=20
lr=0.1
epoch_num=2
train_data_num=100
train_total=6000
small_num=900
iter_num=int(train_data_num/bs)
criterion1 =nn.CrossEntropyLoss()
criterion2 =nn.MSELoss()

start_time = time.time()
for epoch in range(epoch_num):
    print("lr=",lr)
    for step in range(int(train_total/train_data_num)):
        if step>0 and step%5==0 and lr>0.0002:
            del optimizer
            lr=lr/2
            print("lr=",lr)
        optimizer=torch.optim.SGD(net.parameters(),lr)
        
        # read in the data
        train_data = readin_v5.read_part_image(train_data_dir,step*100,100)
        train_label= readin_v5.read_part_xml(train_label_dir,step*100,100)
        print(train_data.size(),train_data.type())
        #train_label=train_label[:,0]
        
        accu_count=0
        step_loss=0.0
        iou_count=0.0
        for iter in range(iter_num):
            if iter>0 :
                del indices
                del minibatch_data
                del minibatch_label
                del inputs
                del prob
                #del log_prob
                del loss
                
            # create a minibatch
            indices=iter*bs
            minibatch_data = train_data[indices:indices+bs]
            minibatch_label= train_label[indices:indices+bs]
            #print(minibatch_label.type())
    
            # feed the input to the net
            inputs=minibatch_data
            inputs.requires_grad_()
            prob=net(inputs)
    
            # update the weights
            #log_prob=torch.log(prob)
            #loss=criterion(log_prob, minibatch_label)
            loss=criterion1(prob[:,0:80], minibatch_label[:,0])+criterion2(prob[:,80:84]/10, minibatch_label[:,1:5].float()/100)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            step_loss=step_loss+loss.item()
            
            # get the accuracy
            accu_batch=get_accu(prob,minibatch_label,bs)
            accu_count=accu_count+accu_batch
            iou_batch=get_iou(prob[:,80:84]/10,minibatch_label[:,1:5].float()/100,bs)
            iou_count=iou_count+iou_batch
            print("batch no.",step*iter_num+iter," loss:",'{0:.2f}'.format(loss.item()),
                  "time:",'{0:.2f}'.format(time.time()-start_time)," accu=",accu_batch," iou=",'{0:.4f}'.format(iou_batch))
        print("step loss: ",'{0:.2f}'.format(step_loss/iter_num),"accu=",'{0:.2f}'.format(accu_count/train_data_num),
              " iou="+'{0:.4f}'.format(iou_count/iter_num))
    del train_data
    del train_label
    del indices
    del minibatch_data
    del minibatch_label
    del inputs
    del prob
    del loss
    
    # save the net data
    net_name=origname+"_epoch_"+str(epoch)+".pt"
    torch.save(net.state_dict(),net_name)
    
    small_loss=0.0
    small_accu=0
    small_iou=0.0
    for step in range(int(small_num/100)):
    
        # read in the data
        small_data = readin_v5.read_part_image(small_data_dir,step*100,100)
        small_label= readin_v5.read_part_xml(small_label_dir,step*100,100)
        print("test:",small_data.size(),small_data.type())
        
        accu_count=0
        step_loss=0.0
        iou_count=0.0
        for iter in range(iter_num):
            if iter>0 :
                del indices
                del minibatch_data
                del minibatch_label
                del prob
                del loss
                del prob_xy
                del real_xy
                
            # create a minibatch
            indices=iter*bs
            minibatch_data = small_data[indices:indices+bs]
            minibatch_label= small_label[indices:indices+bs]
            #print(minibatch_label.type())
    
            # feed the input to the net
            prob=net(minibatch_data)
            loss=criterion1(prob[:,0:80], minibatch_label[:,0])+criterion2(prob[:,80:84]/10, minibatch_label[:,1:5].float()/100)
            step_loss=step_loss+loss.item()
            
            # get the accuracy
            accu_batch=get_accu(prob,minibatch_label,bs)
            accu_count=accu_count+accu_batch
            prob_xy=prob[:,80:84]/10
            real_xy=minibatch_label[:,1:5].float()/100
            iou_batch=get_iou(prob_xy,real_xy,bs)
            iou_item=iou_batch.item()
            del iou_batch
            iou_count=iou_count+iou_item
            print("small_batch no.",step*iter_num+iter," loss:",'{0:.2f}'.format(loss.item()),
                  "time:",'{0:.2f}'.format(time.time()-start_time)," accu=",accu_batch," iou=",'{0:.4f}'.format(iou_item))
        small_loss=small_loss+step_loss
        small_accu=small_accu+accu_count
        small_iou=small_iou+iou_count
        print("small_step loss: ",'{0:.2f}'.format(step_loss/iter_num),"accu=",'{0:.2f}'.format(accu_count/train_data_num),
              " iou="+'{0:.4f}'.format(iou_count/iter_num))
    del small_data
    del small_label
    del indices
    del minibatch_data
    del minibatch_label
    del prob
    del loss
    del prob_xy
    del real_xy
    print("small loss:",'{0:.2f}'.format(small_loss/(small_num/bs)),"accu=",'{0:.2f}'.format(small_accu/small_num),
          " iou="+'{0:.4f}'.format(small_iou/(small_num/bs)))
    

lr= 0.1
torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no. 0  loss: 7.43 time: 7.05  accu= 2  iou= 0.0000
batch no. 1  loss: 6.90 time: 13.26  accu= 0  iou= 0.0000
batch no. 2  loss: 6.68 time: 19.35  accu= 0  iou= 0.0000
batch no. 3  loss: 6.74 time: 25.51  accu= 2  iou= 0.0000
batch no. 4  loss: 5.81 time: 31.64  accu= 0  iou= 0.0000
step loss:  6.71 accu= 0.04  iou=0.0000
torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no. 5  loss: 6.74 time: 38.55  accu= 0  iou= 0.0000
batch no. 6  loss: 4.98 time: 44.83  accu= 3  iou= 0.0000
batch no. 7  loss: 8.53 time: 50.89  accu= 1  iou= 0.0004
batch no. 8  loss: 6.75 time: 57.02  accu= 0  iou= 0.0783
batch no. 9  loss: 7.46 time: 63.15  accu= 0  iou= 0.0301
step loss:  6.89 accu= 0.04  iou=0.0218
torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no. 10  loss: 7.70 time: 70.08  accu= 2  iou= 0.0000
batch no. 11  loss: 7.55 time: 76.13  accu= 0  iou= 0.0000
batch no. 12  loss: 5.85 time: 82.23  accu= 1  iou= 0.0092
batch no.

torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no. 105  loss: 1.28 time: 686.67  accu= 17  iou= 0.0300
batch no. 106  loss: 0.98 time: 692.92  accu= 16  iou= 0.1313
batch no. 107  loss: 0.87 time: 698.86  accu= 18  iou= 0.1233
batch no. 108  loss: 0.75 time: 704.87  accu= 18  iou= 0.0737
batch no. 109  loss: 0.95 time: 710.97  accu= 19  iou= 0.0112
step loss:  0.97 accu= 0.88  iou=0.0739
torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no. 110  loss: 0.86 time: 719.12  accu= 16  iou= 0.0305
batch no. 111  loss: 0.92 time: 725.17  accu= 18  iou= 0.1085
batch no. 112  loss: 0.77 time: 731.19  accu= 17  iou= 0.1324
batch no. 113  loss: 0.93 time: 737.28  accu= 18  iou= 0.0833
batch no. 114  loss: 1.04 time: 743.39  accu= 17  iou= 0.0260
step loss:  0.90 accu= 0.86  iou=0.0761
torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no. 115  loss: 1.35 time: 751.60  accu= 16  iou= 0.1356
batch no. 116  loss: 1.52 time: 757.60  accu= 13  iou= 0.0955
batch no. 117  loss: 1.16 tim

batch no. 206  loss: 0.68 time: 1340.96  accu= 18  iou= 0.0714
batch no. 207  loss: 0.96 time: 1346.98  accu= 16  iou= 0.1074
batch no. 208  loss: 0.83 time: 1353.08  accu= 17  iou= 0.0237
batch no. 209  loss: 0.95 time: 1359.14  accu= 17  iou= 0.0328
step loss:  0.81 accu= 0.87  iou=0.0700
torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no. 210  loss: 0.82 time: 1367.34  accu= 18  iou= 0.0580
batch no. 211  loss: 0.82 time: 1373.37  accu= 18  iou= 0.0930
batch no. 212  loss: 0.87 time: 1379.39  accu= 17  iou= 0.0575
batch no. 213  loss: 0.77 time: 1385.43  accu= 18  iou= 0.0651
batch no. 214  loss: 1.01 time: 1391.42  accu= 17  iou= 0.0357
step loss:  0.86 accu= 0.88  iou=0.0619
torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no. 215  loss: 0.89 time: 1399.50  accu= 18  iou= 0.0573
batch no. 216  loss: 0.96 time: 1405.53  accu= 17  iou= 0.0584
batch no. 217  loss: 1.04 time: 1411.66  accu= 17  iou= 0.0716
batch no. 218  loss: 0.89 time: 1417.75  accu= 16  iou= 0.0482
bat

small_batch no. 7  loss: 1.12 time: 1963.86  accu= 15  iou= 0.1287
small_batch no. 8  loss: 0.91 time: 1966.39  accu= 17  iou= 0.0696
small_batch no. 9  loss: 1.09 time: 1969.14  accu= 18  iou= 0.0222
small_step loss:  0.98 accu= 0.84  iou=0.1013
test: torch.Size([100, 3, 360, 640]) torch.FloatTensor
small_batch no. 10  loss: 0.56 time: 1972.65  accu= 20  iou= 0.0752
small_batch no. 11  loss: 1.05 time: 1975.39  accu= 17  iou= 0.0854
small_batch no. 12  loss: 0.63 time: 1978.00  accu= 18  iou= 0.0907
small_batch no. 13  loss: 0.78 time: 1980.66  accu= 19  iou= 0.0818
small_batch no. 14  loss: 0.98 time: 1983.28  accu= 14  iou= 0.0931
small_step loss:  0.80 accu= 0.88  iou=0.0853
test: torch.Size([100, 3, 360, 640]) torch.FloatTensor
small_batch no. 15  loss: 1.02 time: 1986.59  accu= 17  iou= 0.1098
small_batch no. 16  loss: 0.87 time: 1989.31  accu= 19  iou= 0.0323
small_batch no. 17  loss: 0.85 time: 1991.97  accu= 17  iou= 0.0822
small_batch no. 18  loss: 0.64 time: 1994.70  accu= 1

batch no. 60  loss: 0.64 time: 2448.75  accu= 19  iou= 0.0233
batch no. 61  loss: 0.69 time: 2454.73  accu= 18  iou= 0.0755
batch no. 62  loss: 0.67 time: 2460.85  accu= 19  iou= 0.0524
batch no. 63  loss: 0.57 time: 2466.88  accu= 20  iou= 0.0398
batch no. 64  loss: 1.14 time: 2473.04  accu= 17  iou= 0.0388
step loss:  0.74 accu= 0.93  iou=0.0460
torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no. 65  loss: 0.67 time: 2479.87  accu= 19  iou= 0.1061
batch no. 66  loss: 0.52 time: 2485.96  accu= 20  iou= 0.0914
batch no. 67  loss: 0.67 time: 2492.01  accu= 18  iou= 0.0578
batch no. 68  loss: 0.59 time: 2497.99  accu= 18  iou= 0.0868
batch no. 69  loss: 0.63 time: 2504.05  accu= 19  iou= 0.0265
step loss:  0.61 accu= 0.94  iou=0.0737
torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no. 70  loss: 0.83 time: 2510.89  accu= 20  iou= 0.0710
batch no. 71  loss: 0.73 time: 2517.11  accu= 18  iou= 0.0940
batch no. 72  loss: 0.77 time: 2523.25  accu= 18  iou= 0.0379
batch no. 73  lo

batch no. 163  loss: 0.70 time: 3093.02  accu= 18  iou= 0.1106
batch no. 164  loss: 1.07 time: 3099.12  accu= 14  iou= 0.0408
step loss:  0.84 accu= 0.86  iou=0.1075
torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no. 165  loss: 0.68 time: 3105.98  accu= 18  iou= 0.1370
batch no. 166  loss: 0.66 time: 3112.17  accu= 17  iou= 0.0883
batch no. 167  loss: 1.10 time: 3118.32  accu= 17  iou= 0.1184
batch no. 168  loss: 0.69 time: 3124.44  accu= 17  iou= 0.0439
batch no. 169  loss: 1.01 time: 3130.44  accu= 17  iou= 0.0810
step loss:  0.83 accu= 0.86  iou=0.0937
torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no. 170  loss: 0.82 time: 3137.48  accu= 19  iou= 0.1302
batch no. 171  loss: 1.15 time: 3143.57  accu= 15  iou= 0.0397
batch no. 172  loss: 1.20 time: 3149.75  accu= 14  iou= 0.0421
batch no. 173  loss: 0.91 time: 3155.86  accu= 17  iou= 0.0642
batch no. 174  loss: 0.61 time: 3161.98  accu= 18  iou= 0.1085
step loss:  0.94 accu= 0.83  iou=0.0769
torch.Size([100, 3, 360, 6

torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no. 265  loss: 0.93 time: 3730.67  accu= 18  iou= 0.0652
batch no. 266  loss: 1.15 time: 3736.79  accu= 16  iou= 0.1258
batch no. 267  loss: 1.32 time: 3742.95  accu= 16  iou= 0.0671
batch no. 268  loss: 0.56 time: 3749.01  accu= 19  iou= 0.0588
batch no. 269  loss: 0.77 time: 3755.17  accu= 18  iou= 0.1199
step loss:  0.95 accu= 0.87  iou=0.0874
torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no. 270  loss: 0.51 time: 3761.96  accu= 20  iou= 0.0889
batch no. 271  loss: 1.09 time: 3767.99  accu= 15  iou= 0.0441
batch no. 272  loss: 1.15 time: 3774.04  accu= 17  iou= 0.0751
batch no. 273  loss: 0.51 time: 3780.13  accu= 19  iou= 0.0323
batch no. 274  loss: 1.03 time: 3786.19  accu= 17  iou= 0.1925
step loss:  0.86 accu= 0.88  iou=0.0866
torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no. 275  loss: 0.64 time: 3793.00  accu= 20  iou= 0.0913
batch no. 276  loss: 0.86 time: 3799.11  accu= 18  iou= 0.0667
batch no. 277  lo

In [8]:
print(train_label[0:10])

tensor([[ 69, 268,   0,  75, 171],
        [ 26, 292, 213,  22,  22],
        [  4, 354, 221,  44,  55],
        [ 32, 296, 188,   9,  40],
        [ 49, 261, 158,  11,  40],
        [ 53, 329, 159,  23,  44],
        [ 53, 291, 158,  23,  51],
        [ 18, 251, 169,  38,  38],
        [  2, 296, 105, 126, 187],
        [ 32, 340, 208,   9,  40]])


In [ ]:
loss = nn.MSELoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5)
print(target)
#output = loss(input, target)
xy=prob[:,80:84]/100
xytrue=minibatch_label[:,1:5].float()/100
print(xytrue)
print(xytrue.type(),xytrue.size())
output = loss(xy,xytrue)
print(output.size())


In [ ]:
def cal_iou(infer,target):
    x=infer[0]
    y=infer[1]
    w=infer[2]
    h=infer[3]
    xt=target[0]
    yt=target[1]
    wt=target[2]
    ht=target[3]
    # case 0: when contained by the target
    if ((x+w)>=xt) and ((y+h)>=yt) and ((x+w)<=xt+wt) and ((y+h)<=yt+ht) 
    and (x>=xt) and (y>=yt) and (x<=xt+wt) and (y<=yt+ht):
        same=w*h
        iou=same/(wt*ht)
    # case 1: when right upper part is the same
    elif ((x+w)>=xt) and ((y+h)>=yt) and ((x+w)<=xt+wt) and ((y+h)<=yt+ht):
        same=(x+w-xt)*(y+h-yt)
        iou=same/(w*h+wt*ht-same)
    # case 2: right down part
    elif ((x+w)>=xt) and (y>=yt) and ((x+w)<=xt+wt) and (y<=yt+ht):
        same=(x+w-xt)*(yt+ht-y)
        iou=same/(w*h+wt*ht-same)
    # case 3: left upper part
    elif (x>=xt) and ((y+h)>=yt) and (x<=xt+wt) and ((y+h)<=yt+ht):
        same=(xt+wt-x)*(y+h-yt)
        iou=same/(w*h+wt*ht-same)
    # case 4: left down part
    elif (x>=xt) and (y>=yt) and (x<=xt+wt) and (y<=yt+ht):
        same=(xt+wt-x)*(yt+ht-y)
        iou=same/(w*h+wt*ht-same)
    # case 5: contain the target
    elif ((xt+wt)>=x) and ((yt+ht)>=y) and ((xt+wt)<=x+w) and ((yt+ht)<=y+h) 
    and (xt>=x) and (yt>=y) and (xt<=x+w) and (yt<=y+h):
        same=wt*ht
        iou=same/(w*h)
    else :
        same=0.0
        iou=0.0
    del x,y,w,h,xt,yt,wt,ht,same
    return iou